# Haiku aus dem Nationalrat


protokolle laden und flatten

In [28]:
import json
import pandas as pd

with open("woswormeileistung/data/sessions.json") as f:
    sessions = json.load(f)

wortmeldungen = pd.json_normalize(
    sessions,
    record_path=["sections"],
    meta=["period", "sessionNumber", "date"],
)

wortmeldungen = wortmeldungen[["period", "sessionNumber", "date", "speaker", "text"]]

wortmeldungen.head()

,period,sessionNumber,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
1,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
2,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Der Herr Bund...
3,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Die Amtlichen...
4,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Ich darf beka...


sprecher:in label aus text entfernen

In [29]:
wortmeldungen["text"] = wortmeldungen["text"].str.split(": ", n=1).str[1]
wortmeldungen.head()

,period,sessionNumber,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren Abgeordne...
1,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren auf der G...
2,XXVII,276,2024-09-18T00:00:00,88386,Der Herr Bundespräsident hat mit Entschließung...
3,XXVII,276,2024-09-18T00:00:00,88386,Die Amtlichen Protokolle der 272. und der 273....
4,XXVII,276,2024-09-18T00:00:00,88386,"Ich darf bekannt geben, dass von der Bundeswah..."


dev mode: probelauf auf 1000 texte

In [30]:
wortmeldungen = wortmeldungen.sample(1000)

haiku finden

In [31]:
import spacy
import pyphen
import re
from tqdm import tqdm

!python -m spacy download de_core_news_sm

nlp = spacy.load("de_core_news_sm")
dic = pyphen.Pyphen(lang="de_DE")

def extract_haiku(sentences):
    haiku_in_sentences = []

    for s in sentences:
        s = s.strip()
        s = s.replace(",", "").replace(".", "").replace("!", "").replace("?", "").replace("-", "")
        if not re.fullmatch(r"[A-Za-z ]+", s):
            continue
        
        words = s.split()
        syllable_counts = [dic.inserted(w).count("-") + 1 for w in words]

        line_limits = [5, 7, 5]
        line_idx = 0
        line_sum = 0
        haiku_in_sentence = []

        for word, syllable_count in zip(words, syllable_counts):
            if line_idx >= 3 or line_sum > line_limits[line_idx]:
                break

            if line_sum < line_limits[line_idx]:
                haiku_in_sentence.append(word)
                line_sum += syllable_count
                if line_sum == line_limits[line_idx]:
                    line_idx += 1
                    line_sum = 0
                    if line_idx < 3:
                        haiku_in_sentence.append("-")

        if line_idx == 3 and line_sum == 0 and len(haiku_in_sentence) == len(words) + 2:
            haiku_in_sentences.append(" ".join(haiku_in_sentence))
        
    return haiku_in_sentences


haiku = []
for doc in tqdm(nlp.pipe(wortmeldungen["text"].fillna("").astype(str), batch_size=50), total=len(wortmeldungen)):
    sentences = [sent.text for sent in doc.sents]
    haiku.append(extract_haiku(sentences))

wortmeldungen["haiku"] = haiku
wortmeldungen.head()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.8.0/de_core_news_sm-3.8.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


100%|██████████| 1000/1000 [00:49<00:00, 20.34it/s]


,period,sessionNumber,date,speaker,text,haiku
161966,XXII,125,2005-10-19T00:00:00,8209,Sehr geehrte Frau Präsidentin! Sehr geehrter H...,[]
70082,XXV,121,2016-04-18T00:00:00,145,"Meine sehr geehrten Damen und Herren, da die R...",[]
109040,XXIV,135,2011-12-06T00:00:00,2849,Frau Präsidentin! Hohes Haus! Kollege Obernost...,[]
107251,XXIV,144,2012-02-29T00:00:00,2723,Herr Präsident! Sehr geehrte Frau Finanzminist...,[]
135018,XXIV,19,2009-04-21T00:00:00,35468,Herr Präsident! Sehr geehrter Herr Minister! S...,[]


liste auf haikus flatten

In [32]:
haiku = wortmeldungen.explode("haiku").dropna(subset=["haiku"])
haiku = haiku.rename(columns={"text": "context"})
haiku.head()

,period,sessionNumber,date,speaker,context,haiku
18558,XXVII,167,2022-07-06T00:00:00,5654,Frau Präsidentin! Sehr geehrte Frau Staatssekr...,Ich glaube das ist - ein ganz wichtiges Signal...
134483,XXIV,20,2009-04-22T00:00:00,8240,Frau Präsidentin! Geschätzte Damen und Herren ...,Aus unserer Sicht - wird zu wenig investiert -...
26258,XXVII,119,2021-07-19T00:00:00,6502,Frau Präsidentin! Geschätzte Mitglieder der Bu...,Sie kontrollieren - nicht sie zeichnen nicht a...
174292,XXII,66,2004-06-16T00:00:00,39,Herr Präsident! Herr Staatssekretär! Ich darf ...,Die verbleibenden - Ziffern sind daher neu dur...
82863,XXV,51,2014-11-20T00:00:00,1210,Werte Kolleginnen und Kollegen! Normalerweise ...,und ich frage jetzt - mich und auch ihn warum ...


personen verknüpfen

In [33]:
personen = pd.read_json("woswormeileistung/data/persons.json")
personen = personen[["id", "name", "parties", "imageUrl"]]
personen["id"] = personen["id"].astype(str)
haiku = haiku.merge(personen, left_on="speaker", right_on="id", how="left")
haiku = haiku.drop(columns=["id", "speaker"])
haiku.head()

,period,sessionNumber,date,context,haiku,name,parties,imageUrl
0,XXVII,167,2022-07-06T00:00:00,Frau Präsidentin! Sehr geehrte Frau Staatssekr...,Ich glaube das ist - ein ganz wichtiges Signal...,Dr. Elisabeth Götze,[GRÜNE],https://parlament.gv.at/dokument/bild/200696/2...
1,XXIV,20,2009-04-22T00:00:00,Frau Präsidentin! Geschätzte Damen und Herren ...,Aus unserer Sicht - wird zu wenig investiert -...,Dr. Eva Glawischnig-Piesczek,[GRÜNE],https://parlament.gv.at/dokument/bild/63466/63...
2,XXVII,119,2021-07-19T00:00:00,Frau Präsidentin! Geschätzte Mitglieder der Bu...,Sie kontrollieren - nicht sie zeichnen nicht a...,Mag. Agnes Sirkka Prammer,[GRÜNE],https://parlament.gv.at/dokument/bild/201433/2...
3,XXII,66,2004-06-16T00:00:00,Herr Präsident! Herr Staatssekretär! Ich darf ...,Die verbleibenden - Ziffern sind daher neu dur...,Jakob Auer,[ÖVP],https://parlament.gv.at/dokument/bild/44445/44...
4,XXV,51,2014-11-20T00:00:00,Werte Kolleginnen und Kollegen! Normalerweise ...,und ich frage jetzt - mich und auch ihn warum ...,Dr. Peter Pilz,"[GRÜNE, Ohne Klub, PILZ]",https://parlament.gv.at/dokument/bild/84796/84...


stabile haiku id aus sessionnumber+person+haiku erstellen

In [35]:
# todo

exportieren

In [34]:
haiku.to_json("haiku.json", orient="records", force_ascii=False, indent=2)